# Experiments

Running this notebook reproduces the experiments from the paper.

In [1]:
import math
import gudhi
import numpy as np
from miniball import Miniball
import timeit
import functools
import warnings
warnings.filterwarnings('ignore')

from core import core_alpha, sqrt_persistence, persistence_intervals_in_dimension
from datasets import sample_circle, sample_rectangle, sample_torus, sample_cube, sample_flat_torus, sample_sphere

## Helper Functions

Compute alpha-core persistent homology from a point cloud $X$ for multiple values of $s_\text{max}$.

In [2]:
def persistence_diagrams(
    X, max_ss: list[int] = [1, 10, 100, 1000], 
    max_r: float | None = -1
) -> list[tuple[int, tuple[int, int]]]:
    if max_r < 0:
        max_r = 2*math.sqrt(Miniball(X).squared_radius())
    res = []
    for i, max_s in enumerate(max_ss):
        max_k = max(1, int((M + N) * max_s))
        st = core_alpha(X, max_k=max_k, max_r=max_r)
        persistence = sqrt_persistence(st)
        res.append(persistence)
    return res

Compute the bottleneck distance (element-wise) between two lists of persistence diagrams in a given homological dimension.

In [3]:
def bottleneck_distances(pers1, pers2, dim):
    assert len(pers1) == len(pers2)
    n = len(pers1)
    A = np.zeros(n)
    for i in range(n):
        a = persistence_intervals_in_dimension(pers1[i], dim)
        b = persistence_intervals_in_dimension(pers2[i], dim)
        bdist = gudhi.bottleneck_distance(a, b)
        A[i] = bdist
    return A

### Point Cloud Dataset Generators

Functions for generating the datasets.

In [4]:
def circle_with_noise(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_circle(N, rng, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]    

In [5]:
def two_circles_with_noise(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    N1 = (2 * N) // 3
    N2 = N // 3 
    
    Z1 = sample_circle(N1, rng, r=1, std=sigma)
    Z2 = sample_circle(N2, rng, r=0.5, std=sigma)
    Z = np.r_[Z1, Z2]
    
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z1, Z2, Y]

In [6]:
def embedded_torus(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_torus(N, rng, a=1, b=3, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]

In [7]:
def sphere(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_sphere(N, rng, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]

In [8]:
def clifford_torus(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_flat_torus(N, rng, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]

### Bottleneck Distances

The following function compute bottleneck distances between the alpha-core persistence and the ground truth diagrams for a given point cloud dataset generator.

In [9]:
def bottleneck_distance_experiment(
    Ms, Ns, max_ss=[0, 0.001, 0.01, 0.1], sigma=0.07, max_r=-1, 
    point_generator=circle_with_noise, seed=0):
    rng = np.random.default_rng(seed=seed)
    X = point_generator(0, 5, sigma=0, rng=rng)
    dimensions = range(X.shape[1])
    res = {dim: [] for dim in dimensions}
    for M, N in zip(Ms, Ns):
        print(f"M={M}, N={N}")
        X = point_generator(0, M + N, sigma=0, rng=rng)
        st_ideal = core_alpha(X, max_k=1, max_r=max_r)
        persistence_ideal = sqrt_persistence(st_ideal)    
        for max_s in max_ss:
            X = point_generator(M, N, sigma, rng=rng)
            if max_r is not None:
                max_r = 2*math.sqrt(Miniball(X).squared_radius())
            max_k = max(1, int((M + N) * max_s))
            print(f"\tmax_s={max_s} (max_k = {max_k})")
            st = core_alpha(X, max_k=max_k, max_r=max_r)
            persistence = sqrt_persistence(st)
            for dim in dimensions:
                res[dim].append(
                    gudhi.bottleneck_distance(
                        persistence_intervals_in_dimension(persistence, dim),
                        persistence_intervals_in_dimension(persistence_ideal, dim)))
    return res
        
        

Helper function for printing the results.

In [10]:
def formatted_bottleneck_results(bottleneck_distances, Ms, Ns, max_ss):
    res = [f"Ns={Ns} Ms={Ms} max_ss={max_ss}"]
    for dim in bottleneck_distances.keys():
        res.append(f"Dim {dim} & " + " & ".join([f"{dist:.3f}" for dist in bottleneck_distances[dim]]))
    return res

Function for running the experiments with a list of different point cloud dataset generators and a list of $s_\text{max}$.

In [11]:
def run_experiments(
    point_generators,
    Ms,
    Ns,
    max_ss,
    names=None,
    sigma=0.07,
    seed=0,
    max_r=-1,
):
    res = []
    for idx, generator in enumerate(point_generators):
        if names is not None:
            name = names[idx]
        else:
            name = 'Unknown'
        print(f"Running experiments for {name}")
        bottleneck_dists = bottleneck_distance_experiment(
            Ms = Ms, Ns=Ns, max_ss=max_ss, point_generator=generator, sigma=sigma, seed=seed, max_r=max_r)
        res.append([name] +
                   formatted_bottleneck_results(bottleneck_dists, Ms, Ns, max_ss))
    return res

## Run Experiments
### Experiment Settings

In [12]:
Ms = [10_000, 1_000, 100]
Ns = [10_000, 10_000, 10_000]
max_ss=[0, 0.001, 0.01, 0.1]
sigma = 0.07
seed = 0
point_generators = [circle_with_noise, two_circles_with_noise, sphere, embedded_torus, clifford_torus]
names = ["Circle", "Circles", "Sphere", "Torus 1", "Torus 2"]

### Persistence along a line

Run experiments for computing alpha-core persistence along a line.

In [13]:
%%time
result = run_experiments(
    point_generators,
    Ms = Ms,
    Ns = Ns,
    max_ss = max_ss,
    sigma=sigma,
    seed=seed,
    names = names)

Running experiments for Circle
M=10000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 20)
	max_s=0.01 (max_k = 200)
	max_s=0.1 (max_k = 2000)
M=1000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 11)
	max_s=0.01 (max_k = 110)
	max_s=0.1 (max_k = 1100)
M=100, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 10)
	max_s=0.01 (max_k = 101)
	max_s=0.1 (max_k = 1010)
Running experiments for Circles
M=10000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 20)
	max_s=0.01 (max_k = 200)
	max_s=0.1 (max_k = 2000)
M=1000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 11)
	max_s=0.01 (max_k = 110)
	max_s=0.1 (max_k = 1100)
M=100, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 10)
	max_s=0.01 (max_k = 101)
	max_s=0.1 (max_k = 1010)
Running experiments for Sphere
M=10000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 20)
	max_s=0.01 (max_k = 200)
	max_s=0.1 (max_k = 2000)
M=1000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 11)
	max_s=0.01 (max_k = 110)
	max_s=0.1 

Print the results.

In [14]:
print('\n'.join(['\n'.join(x) for x in result]))

Circle
Ns=[10000, 10000, 10000] Ms=[10000, 1000, 100] max_ss=[0, 0.001, 0.01, 0.1]
Dim 0 & 0.012 & 0.014 & 0.073 & 0.342 & 0.034 & 0.020 & 0.053 & 0.288 & 0.106 & 0.010 & 0.051 & 0.275
Dim 1 & 0.499 & 0.499 & 0.499 & 0.432 & 0.499 & 0.499 & 0.499 & 0.328 & 0.499 & 0.499 & 0.270 & 0.308
Circles
Ns=[10000, 10000, 10000] Ms=[10000, 1000, 100] max_ss=[0, 0.001, 0.01, 0.1]
Dim 0 & 0.125 & 0.125 & 0.096 & 0.357 & 0.125 & 0.114 & 0.092 & 0.340 & 0.064 & 0.125 & 0.099 & 0.341
Dim 1 & 0.249 & 0.249 & 0.249 & 0.249 & 0.248 & 0.248 & 0.162 & 0.248 & 0.248 & 0.248 & 0.223 & 0.248
Sphere
Ns=[10000, 10000, 10000] Ms=[10000, 1000, 100] max_ss=[0, 0.001, 0.01, 0.1]
Dim 0 & 0.042 & 0.073 & 0.220 & 0.595 & 0.091 & 0.055 & 0.186 & 0.548 & 0.146 & 0.049 & 0.175 & 0.544
Dim 1 & 0.029 & 0.016 & 0.016 & 0.016 & 0.044 & 0.020 & 0.020 & 0.020 & 0.020 & 0.022 & 0.022 & 0.022
Dim 2 & 0.475 & 0.475 & 0.475 & 0.475 & 0.470 & 0.470 & 0.340 & 0.470 & 0.465 & 0.423 & 0.283 & 0.465
Torus 1
Ns=[10000, 10000, 10000] Ms=

### Persistence for fixed $s$ (and $k$)

Run experiments for computing alpha-core persistence for a fixed $s$.

In [15]:
%%time
result_fixed_k = run_experiments(
    point_generators,
    Ms = Ms,
    Ns = Ns,
    max_ss = max_ss,
    sigma=sigma,
    seed=seed,
    names = names,
    max_r=None,
)

Running experiments for Circle
M=10000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 20)
	max_s=0.01 (max_k = 200)
	max_s=0.1 (max_k = 2000)
M=1000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 11)
	max_s=0.01 (max_k = 110)
	max_s=0.1 (max_k = 1100)
M=100, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 10)
	max_s=0.01 (max_k = 101)
	max_s=0.1 (max_k = 1010)
Running experiments for Circles
M=10000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 20)
	max_s=0.01 (max_k = 200)
	max_s=0.1 (max_k = 2000)
M=1000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 11)
	max_s=0.01 (max_k = 110)
	max_s=0.1 (max_k = 1100)
M=100, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 10)
	max_s=0.01 (max_k = 101)
	max_s=0.1 (max_k = 1010)
Running experiments for Sphere
M=10000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 20)
	max_s=0.01 (max_k = 200)
	max_s=0.1 (max_k = 2000)
M=1000, N=10000
	max_s=0 (max_k = 1)
	max_s=0.001 (max_k = 11)
	max_s=0.01 (max_k = 110)
	max_s=0.1 

Print the results.

In [16]:
print('\n'.join(['\n'.join(x) for x in result_fixed_k]))

Circle
Ns=[10000, 10000, 10000] Ms=[10000, 1000, 100] max_ss=[0, 0.001, 0.01, 0.1]
Dim 0 & 0.012 & 0.014 & 0.074 & 0.371 & 0.034 & 0.020 & 0.053 & 0.312 & 0.106 & 0.010 & 0.052 & 0.301
Dim 1 & 0.499 & 0.499 & 0.499 & 0.393 & 0.499 & 0.499 & 0.499 & 0.357 & 0.499 & 0.499 & 0.263 & 0.338
Circles
Ns=[10000, 10000, 10000] Ms=[10000, 1000, 100] max_ss=[0, 0.001, 0.01, 0.1]
Dim 0 & 0.125 & 0.125 & 0.092 & 0.377 & 0.125 & 0.114 & 0.088 & 0.356 & 0.064 & 0.125 & 0.098 & 0.356
Dim 1 & 0.249 & 0.249 & 0.249 & 0.249 & 0.248 & 0.248 & 0.159 & 0.248 & 0.248 & 0.248 & 0.219 & 0.248
Sphere
Ns=[10000, 10000, 10000] Ms=[10000, 1000, 100] max_ss=[0, 0.001, 0.01, 0.1]
Dim 0 & 0.042 & 0.073 & 0.227 & 0.635 & 0.091 & 0.055 & 0.190 & 0.589 & 0.146 & 0.049 & 0.177 & 0.590
Dim 1 & 0.029 & 0.016 & 0.016 & 0.016 & 0.044 & 0.020 & 0.020 & 0.020 & 0.020 & 0.022 & 0.022 & 0.022
Dim 2 & 0.475 & 0.475 & 0.475 & 0.475 & 0.470 & 0.470 & 0.302 & 0.470 & 0.465 & 0.423 & 0.278 & 0.465
Torus 1
Ns=[10000, 10000, 10000] Ms=

### Runtime Analysis

Measure the runtime for different values of $k$ and $|X|$.

In [17]:
repeat = 10
Ns = [10000, 20000, 30000, 40000, 50000, 60000]
ks = [10, 100, 1000, 10000]
sigma = 0.07

def alpha_core_persistence(X, k):
    st = core_alpha(X, max_k=k, max_r=None)
    st.persistence()

for k in ks:
    print(f"\nk = {k}")
    for N in Ns:
        X = embedded_torus(N // 2, N // 2, sigma, rng=None, seed=0)
        times = timeit.repeat(functools.partial(alpha_core_persistence, X, k), number=1, repeat=repeat)
        print(f"|X| = {N}, Runtime: {min(times):.2f}s.")


k = 10
|X| = 10000, Runtime: 1.13s.
|X| = 20000, Runtime: 2.41s.
|X| = 30000, Runtime: 3.71s.
|X| = 40000, Runtime: 4.96s.
|X| = 50000, Runtime: 6.48s.
|X| = 60000, Runtime: 7.80s.

k = 100
|X| = 10000, Runtime: 1.18s.
|X| = 20000, Runtime: 2.53s.
|X| = 30000, Runtime: 3.88s.
|X| = 40000, Runtime: 5.24s.
|X| = 50000, Runtime: 6.65s.
|X| = 60000, Runtime: 8.09s.

k = 1000
|X| = 10000, Runtime: 1.75s.
|X| = 20000, Runtime: 3.70s.
|X| = 30000, Runtime: 5.71s.
|X| = 40000, Runtime: 7.77s.
|X| = 50000, Runtime: 10.07s.
|X| = 60000, Runtime: 12.13s.

k = 10000
|X| = 10000, Runtime: 7.98s.
|X| = 20000, Runtime: 17.18s.
|X| = 30000, Runtime: 27.24s.
|X| = 40000, Runtime: 36.28s.
|X| = 50000, Runtime: 49.05s.
|X| = 60000, Runtime: 59.02s.
